<h1 style="background-color:black;
           font-family:segoe ui;
                color:#FFF9ED;
           font-size:300%;
           text-align:center;
           border-radius:10px 10px;"> 
    🌐 תהליך הסבת גושים מוסדרים/לא מוסדרים/מבוטלים לבנק"ל מודרני 🌐

In [1]:
print('1. Blocks Borders:')

1. Blocks Borders:


<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
ספריות 📚

In [2]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import geopandas as gpd
import subprocess

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">קריאת נתונים 📑 

In [3]:
print('  1.a Reading inputs')

  1.a Reading inputs


In [4]:
RawData_folder = r'\\mapi_shares\MNCDB\צוות מידע\RawData/'

<h1 style="text-align:right;font-size:100%">
נתוני שכבת גושים
<h1 style="text-align:right;font-size:100%">
טבלת גושים
<h1 style="text-align:right;font-size:100%">
שכבת מפתח גליונות גושים
<h1 style="text-align:right;font-size:100%">
טבלת גושי ממ"ג מאורקל

In [6]:
SubGush_layer_cols = ['SUB_GUSH_ID', 'GUSH_NUM', 'GUSH_SUFFIX', 'STATUS', 'geometry']
SubGush_layer = gpd.read_file(RawData_folder + 'CADSDE\sub_gush_all.gdb', layer = 'sub_gush_all', driver = 'FileGDB', include_fields = SubGush_layer_cols)


SubGush_table_cols = ['SUB_GUSH_ID', 'GUSH_NUM', 'GUSH_SUFFIX', 'STATUS', 'STATUS_DATE', 'HESDER_LAST_PARCEL']
SubGush_table = pd.read_csv(RawData_folder + 'Oracle\sub_gush.csv', usecols=SubGush_table_cols, encoding='cp1255')


SheetKey_cols = ['GUSH_NUM', 'GUSH_SUFFIX', 'STATUS', 'geometry']
SheetKey = gpd.read_file(RawData_folder + 'CADSDE\SHEET_K.gdb', layer = 'SHEET_K', driver = 'FileGDB', include_fields = SheetKey_cols)


SubMMG_cols = ['SUB_GUSH_ID', 'LAST_PARCEL', 'MMG_LAST_PARCEL', 'COURT_LAST_PARCEL', 'FINAL_CONFIRMATION_DATE']
SubMMG = pd.read_csv(RawData_folder + 'Oracle\sub_gush_mmg.csv', usecols = SubMMG_cols, encoding='cp1255')



del [RawData_folder, SubGush_layer_cols, SubGush_table_cols, SheetKey_cols, SubMMG_cols]

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
עיבוד נתונים 🛠️

In [ ]:
print('  1.b Cleaning data')

<h1 style="text-align:right;font-size:125%">
סינון גושים לפי סטאטוסים מוסדרים/לא מוסדרים/מבוטלים למשתנים שונים
<h1 style="text-align:right;font-size:100%">
הגדרת מספרי סטאטוס
<h1 style="text-align:right;font-size:100%">
גושים מוסדרים
<h1 style="text-align:right;font-size:100%">
גושים לא מוסדרים
<h1 style="text-align:right;font-size:100%">
גושים מבוטלים

In [7]:
register_status   = [1, 3, 6, 16]
unregister_status = [4, 9, 10, 11, 13, 15, 17, 18]
cancel_status     = [5, 51, 56, 57, 58]

Reg_SubGush_layer = SubGush_layer[SubGush_layer['STATUS'].isin(register_status)]
Reg_SubGush_table = SubGush_table[SubGush_table['STATUS'].isin(register_status)]


Unreg_SheetKey = SheetKey[SheetKey['STATUS'].isin(unregister_status)]
Unreg_SubGush_table = SubGush_table[SubGush_table['STATUS'].isin(unregister_status)]


cancel_SubGush_table = SubGush_table[SubGush_table['STATUS'].isin(cancel_status)]

<h1 style="text-align:right;font-size:100%">
מיזוג נתונים של גושים מוסדרים לשכבת אחת

In [8]:
Reg_merge = Reg_SubGush_table.merge(SubMMG, on = 'SUB_GUSH_ID' , how= 'left' , suffixes = ('','_merge'))
Reg_merge = Reg_merge.merge(Reg_SubGush_layer, on = 'SUB_GUSH_ID' , how= 'left' , suffixes = ('','_merge'))
Reg_merge.drop(Reg_merge.filter(regex='_merge$').columns.tolist(),axis=1, inplace=True)

<h1 style="text-align:right;font-size:100%">
מיזוג נתונים של גושים לא מוסדרים לשכבת אחת

In [9]:
Unreg_merge = Unreg_SubGush_table.merge(SubMMG, on = 'SUB_GUSH_ID' , how= 'left' , suffixes = ('','_merge'))

Unreg_merge['GUSH_SUFFIX'].fillna(0, inplace=True)
Unreg_merge['GUSH_SUFFIX'] = Unreg_merge['GUSH_SUFFIX'].astype(int)
Unreg_merge['key_id'] = Unreg_merge['GUSH_NUM'].astype(str) + '_' + Unreg_merge['GUSH_SUFFIX'].astype(str)
Unreg_SheetKey['key_id'] = Unreg_SheetKey['GUSH_NUM'].astype(str) + '_' + Unreg_SheetKey['GUSH_SUFFIX'].astype(str)

Unreg_merge = Unreg_merge.merge(Unreg_SheetKey, on = 'key_id' , how= 'left' , suffixes = ('','_merge'))
Unreg_merge.drop(Unreg_merge.filter(regex='_merge$').columns.tolist(),axis=1, inplace=True)

<h1 style="text-align:right;font-size:100%">
מיזוג נתונים של גושים מבוטלים לשכבת אחת

In [10]:
Cancel_merge = cancel_SubGush_table.merge(SubMMG, on = 'SUB_GUSH_ID' , how= 'left' , suffixes = ('','_merge'))
Cancel_merge.drop(Cancel_merge.filter(regex='_merge$').columns.tolist(),axis=1, inplace=True)

<h1 style="text-align:right;font-size:100%">
תיקון שדה של תאריך סטאטוס בגושים מבוטלים

In [11]:
Cancel_merge['STATUS_DATE'] = pd.to_datetime(Cancel_merge['STATUS_DATE'], errors='coerce')

<h1 style="text-align:right;font-size:100%">
הוספת שדה בינארי של סוג מקרקעין לשלושת שכבות הגושים

In [12]:
RegLandType_cond  = [(Reg_merge['STATUS'].isin(register_status))]
Reg_merge['LandType'] = np.select(RegLandType_cond, [1])


UnregLandType_cond  = [(Unreg_merge['STATUS'].isin(unregister_status))]
Unreg_merge['LandType'] = np.select(UnregLandType_cond, [2])


Cancel_merge['LandType'] = 4

<h1 style="text-align:right;font-size:100%">
הוספת שדה בינארי של גוש ירדני לשלושת שכבות הגושים

In [13]:
Reg_Jordanian_cond  = [(Reg_merge['STATUS'].isin([16])),(~Reg_merge['STATUS'].isin([16]))]
Reg_merge['IsJordanian'] = np.select(Reg_Jordanian_cond, [1,0])

Unreg_Jordanian_cond  = [(Unreg_merge['STATUS'].isin([16])),(~Unreg_merge['STATUS'].isin([16]))]
Unreg_merge['IsJordanian'] = np.select(Unreg_Jordanian_cond, [1,0])

Cancel_merge['IsJordanian'] = 0

<h1 style="text-align:right;font-size:100%">
איחד נתוני גושים מוסדרים ולא מוסדרים

In [14]:
appended = gpd.GeoDataFrame(Reg_merge.append(Unreg_merge, ignore_index=True),
                            crs=2039,
                            geometry='geometry')

<h1 style="text-align:right;font-size:100%">
תיקון והגדרת שדות תאריך כתאריך

In [15]:
appended['STATUS_DATE'].fillna(np.nan, inplace=True)
appended['STATUS_DATE'] = appended['STATUS_DATE'].astype(np.datetime64)


appended['FINAL_CONFIRMATION_DATE'].fillna(np.nan, inplace=True)
appended['FINAL_CONFIRMATION_DATE'] = appended['FINAL_CONFIRMATION_DATE'].astype(np.datetime64)

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
יצירת שכבה חדשה בהתאם לאפיון ✨

In [16]:
print('  1.d Defining new Layer')

  1.d Defining new Layer


In [17]:
new_columns = np.dtype([
                        ("BlockUniqueID", int),
                        ('BlockNumber', int),
                        ("SubBlockNumber", int),
                        ("BlockStatus", int),
                        ("IsJordanian", int),
                        ("LandType", int),
                        ("SetteledDate", np.datetime64),
                        ("UpdateDate", np.datetime64),
                        ("LastRegisterdParcel", int),
                        ("LastParcel", int),
                        ("LastSetteledParcel", int),
                        ("LastCourtParcel", int)
                        ])


Blocks = pd.DataFrame(np.empty(0, dtype=new_columns))

CancelBlocks = pd.DataFrame(np.empty(0, dtype=new_columns))

<h1 style="text-align:right;font-size:100%">
איכלוס שדות לגושים מוסדרים ולא מוסדרים

In [18]:
Blocks['BlockUniqueID'] = appended['SUB_GUSH_ID'].astype(int)


Blocks['BlockNumber'] = appended['GUSH_NUM'].astype(int)


Blocks['SubBlockNumber'] = appended['GUSH_SUFFIX'].fillna(0)
Blocks['SubBlockNumber'] = Blocks['SubBlockNumber'].astype(int)


status_clf = {1:12,
              3:13,
              4:21,
              5:41,
              6:11,
              9:26,
              10:22,
              11:23,
              13:24,
              15:25,
              16:11,
              17:27,
              18:28,
              51:42,
              56:45,
              57:43,
              58:44}
Blocks['BlockStatus'] = appended['STATUS'].map(status_clf)
Blocks['BlockStatus'] = Blocks['BlockStatus'].fillna(11)
Blocks['BlockStatus'] = Blocks['BlockStatus'].astype(int)


Blocks['IsJordanian'] = appended['IsJordanian'].astype(int)


Blocks['LandType'] = appended['LandType'].astype(int)


Blocks['SetteledDate'] = appended['STATUS_DATE'].astype('datetime64[D]')


Blocks['UpdateDate'] = appended['FINAL_CONFIRMATION_DATE'].astype('datetime64[D]')


Blocks['LastRegisterdParcel'] = appended['LAST_PARCEL'].astype('Int64')


Blocks['LastParcel'] = appended['MMG_LAST_PARCEL'].astype('Int64')


Blocks['LastSetteledParcel'] = appended['HESDER_LAST_PARCEL'].astype('Int64')


Blocks['LastCourtParcel'] = appended['COURT_LAST_PARCEL'].astype('Int64')


Blocks['geometry'] = appended['geometry']


Blocks['IsGeom'] = np.where(Blocks['geometry']!= None, 1, 0) #Temporary field for removing duplicates

<h1 style="text-align:right;font-size:100%">
תיקון הגדרת סוג שדה לשדות של תאריכים
<h1 style="text-align:right;font-size:100%">    
מיון הטבלה לפי מספר מזהה גוש

In [19]:
Blocks.sort_values(['BlockUniqueID', 'IsGeom', 'SetteledDate', 'UpdateDate'], inplace=True)

<h1 style="text-align:right;font-size:100%">
קיבוץ שכבה על מנת להסיר כפיוליות

In [20]:
group_agg = {
             'BlockNumber':        'max',
             'SubBlockNumber':     'max',
             'BlockStatus':        'max',
             'IsJordanian':        'max',
             'LandType':           'max',
             'SetteledDate':       'last',
             'UpdateDate':         'last',
             'LastRegisterdParcel':'max',
             'LastParcel':         'max',
             'LastSetteledParcel': 'max',
             'LastCourtParcel':    'max',
             'IsGeom':             'max',
             'geometry':           'last'
            }

Blocks = Blocks.groupby('BlockUniqueID', dropna=True, as_index=False).agg(group_agg)
Blocks.drop(columns='IsGeom', inplace=True)

<h1 style="text-align:right;font-size:150%; color:#1c6ce6;">
איכלוס שדות לגושים מבוטלים

In [21]:
CancelBlocks['BlockUniqueID'] = Cancel_merge['SUB_GUSH_ID'].astype(int)


CancelBlocks['BlockNumber'] = Cancel_merge['GUSH_NUM'].astype(int)


CancelBlocks['SubBlockNumber'] = Cancel_merge['GUSH_SUFFIX'].fillna(0)
CancelBlocks['SubBlockNumber'] = CancelBlocks['SubBlockNumber'].astype(int)


CancelBlocks['BlockStatus'] = None


CancelBlocks['IsJordanian'] = Cancel_merge['IsJordanian']


CancelBlocks['LandType'] = Cancel_merge['LandType']


CancelBlocks['SetteledDate'] = Cancel_merge['STATUS_DATE'].astype('datetime64[D]')


CancelBlocks['UpdateDate'] = Cancel_merge['FINAL_CONFIRMATION_DATE'].astype('datetime64[D]')


CancelBlocks['LastRegisterdParcel'] = CancelBlocks['LastRegisterdParcel'].fillna(0)
CancelBlocks['LastRegisterdParcel'] = Cancel_merge['LAST_PARCEL'].astype('Int64')


CancelBlocks['LastParcel'] = Cancel_merge['MMG_LAST_PARCEL'].astype('Int64')


CancelBlocks['LastSetteledParcel'] = Cancel_merge['HESDER_LAST_PARCEL'].astype('Int64')


CancelBlocks['LastCourtParcel'] = Cancel_merge['COURT_LAST_PARCEL'].fillna(0)
CancelBlocks['LastCourtParcel'] = Cancel_merge['COURT_LAST_PARCEL'].astype('Int64')



CancelBlocks[['SetteledDate','UpdateDate']] = CancelBlocks[['SetteledDate','UpdateDate']].astype(np.datetime64)
CancelBlocks.sort_values('BlockUniqueID', inplace=True)

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
שמירת תוצאות לגושים מוסדרים/לא מוסדרים 💾
<h1 style="text-align:right; font-size:100%;"> 
<h1 style="text-align:right; font-size:100%;">     
פיצול התוצאות לפי יישויות בעלי גיאומטריה וללא גיאומטריה    
<h1 style="text-align:right; font-size:100%;"> 
בחירת נתיב
<h1 style="text-align:right; font-size:100%;"> 
שמירה כטבלה
<h1 style="text-align:right; font-size:100%;"> 
שמירה כשכבה גיאוגרפית

In [ ]:
print('  1.e Exporting results to product folder')

In [ ]:
Blocks_geom = gpd.GeoDataFrame(Blocks[~Blocks['geometry'].isna()],
                               geometry = 'geometry',
                               crs = 2039)


Blocks_no_geom = Blocks[Blocks['geometry'].isna()]

In [ ]:
#geom_csv_dir = r'\\mapi_shares\MNCDB\צוות מידע\תוצרים\csv\Blocks_geom.csv'
no_geom_csv_dir = r'\\mapi_shares\MNCDB\צוות מידע\תוצרים\csv\Blocks_no_geom.csv'
json_dir =  r'\\mapi_shares\MNCDB\צוות מידע\תוצרים\json\Blocks_geom.geojson'


#Blocks_geom.to_csv(geom_csv_dir ,encoding='cp1255', index=False)
Blocks_no_geom.drop(columns='geometry').to_csv(no_geom_csv_dir , index=False)


Blocks_geom.to_file(json_dir, driver = 'GeoJSON')

In [ ]:
subprocess.call([r"M:\ncdbScripts\scripts\arcpyPro\Blocks-Save as Feature Class.bat"]);

<h1 style="text-align:right;font-size:150%; color:#1c6ce6;">
שמירת תוצאות לגושים מבוטלים
<h1 style="text-align:right;font-size:100%">
בחירת נתיב
<h1 style="text-align:right;font-size:100%">
שמירה כטבלה

In [ ]:
CancelBlocks_csv_dir = r'\\mapi_shares\MNCDB\צוות מידע\תוצרים\csv\CancelBlocks.csv'

CancelBlocks.to_csv(CancelBlocks_csv_dir, index=False)

In [ ]:
subprocess.call([r"M:\ncdbScripts\scripts\arcpyPro\CancelBlocks-Save as Feature Class.bat"]);